In [1]:
import warnings
warnings.filterwarnings('ignore')
import picaso.justdoit as jdi
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
T_eff= 1000 # Effective Temperature of your Brown Dwarf in K
grav = 1000 # Gravity of your brown dwarf in m/s/s

cl_run = jdi.inputs(calculation="brown", climate = True) 


cl_run.gravity(gravity=grav, gravity_unit=u.Unit('m/(s**2)')) 
cl_run.T_eff(Teff=T_eff) # input effective temperature



T_star = "NA" # K, As it is a isolated brown dwarf, these params are not needed so 'NA'
semi_major = "NA" # AU,  As it is a isolated brown dwarf, these params are not needed so 'NA'
# ignore this for now
self_consistent_kzz= False # Only needed if you want to do on-the-fly calculation with MLT Kzz



In [2]:
mh = '+0.0' #log metallicity
CtoO = '1.0' # CtoO ratio

# Connect to your opacity databases. These databases are the backbone of the code
filename_db="/Users/sagnickmukherjee/Documents/GitHub/picaso-dev/reference/opacities/ck_cx_cont_opacities.db"
ck_db='/Users/sagnickmukherjee/Documents/software/picaso-dev/reference/opacities/ck_db/m'+mh+'_co'+CtoO+'.data.196'

# Notice The keyword ck is set to True because you want to use the correlated-k opacities for your calculation
# and not the line by line opacities
opacity_ck = jdi.opannection(ck=True, ck_db=ck_db,filename_db=filename_db) # grab your opacities


In [4]:
nlevel = 91 # number of plane-parallel levels in your code

#Lets set the max and min at 1e-4 bars and 500 bars

Pmin = 1e-4 #bars
Pmax = 500 #bars
pressure=np.logspace(np.log10(Pmin),np.log10(Pmax),nlevel) # set your pressure grid


In [5]:
temp_guess = np.zeros(shape=(nlevel))

temp_guess += 500 # K , isothermal atmosphere guess


In [6]:

    
nofczns = 1 # number of convective zones initially, don't change this 

nstr_upper = 73 # top most level of guessed convective zone
nstr_deep = nlevel -2 # this is always the case. Dont change this
nstr = np.array([0,nstr_upper,nstr_deep,0,0,0]) # initial guess of convective zones

# Here are some other parameters needed for the code.
rfacv = 0.0 # This is for stellar incident radiation. This is zero for brown dwarfs as we are not concerned with stellar radiation.
rfaci =1.0 # This is 1 always.
cloudy = False # not cloudy
fsed = 0.0 # as not cloudy so won't matter

    

In [7]:
cl_run.inputs_climate(temp_guess= temp_guess, pressure= pressure, nstr = nstr, nofczns = nofczns , rfacv = rfacv, rfaci = rfaci, cloudy = cloudy, mh = mh, CtoO = CtoO,species = ['MgSiO3','Mg2SiO4','Fe'], fsed = fsed)


In [8]:
pressure , temp, dtdp, nstr_new, flux_plus_final, df, all_profiles = cl_run.run_climate_model(opacity_ck,save_all_profiles=True, save_all_kzz = False,diseq_chem = False, self_consistent_kzz =self_consistent_kzz,vulcan_run = False)


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'climate':[0m [1m[1mcannot determine Numba type of <class 'function'>[0m
[1m
File "picaso/climate.py", line 411:[0m
[1mdef t_start(nofczns,nstr,it_max,conv,x_max_mult, 
    <source elided>
    #both reflected and thermal
[1m    flux_net_v_layer_full, flux_net_v_full, flux_plus_v_full, flux_minus_v_full , flux_net_ir_layer_full, flux_net_ir_full, flux_plus_ir_full, flux_minus_ir_full = climate(pressure, temp, dwni, bb , y2, tp, tmin, tmax, DTAU, TAU, W0, 
[0m    [1m^[0m[0m
[0m

In [13]:
import pandas as pd
Teff = 700
grav = 316
df1 = pd.read_csv("../../research/deq_miles/gridruns_new/profiles_new/profilegrid_eq_qt_eq_grid_"+str(Teff)+"_grav_"+str(grav)+"_mh_+0.0_sm_NA.dat", sep='\t')


bundle = jdi.inputs(calculation="brown")

bundle.phase_angle(0)
bundle.gravity(gravity=grav , gravity_unit=u.Unit('m/s**2'))

sonora_profile_db = '/Users/sagnickmukherjee/Documents/software/sonora/profile/'
Teff_dummy = 1700
bundle.sonora(sonora_profile_db, Teff_dummy)

for keys in df1.keys():
    bundle.inputs['atmosphere']['profile'][keys] = df1[keys]


In [14]:
from picaso.climate import  calculate_atm_deq, did_grad_cp, convec, calculate_atm, t_start, growdown, growup, climate
from picaso.fluxes import set_bb, tidal_flux

DTAU, TAU, W0, COSB,ftau_cld, ftau_ray,GCOS2, DTAU_OG, TAU_OG, W0_OG, COSB_OG, \
        W0_no_raman , surf_reflect, ubar0,ubar1,cos_theta, single_phase,multi_phase, \
        frac_a,frac_b,frac_c,constant_back,constant_forward, tridiagonal , \
        wno,nwno,ng,nt, nlevel, ngauss, gauss_wts, mmw =  calculate_atm(bundle, opacity_ck )

pressure,temp = df1['pressure'],df1['temperature']

extension = 0.3 
tmin = 52.0 #min_temp*(1-extension)
tmax = 5199 #max_temp*(1+extension)
dt = 2.5
ntmps = int((tmax-tmin)/dt)
delta_wno,dwni = opacity_ck.delta_wno,opacity_ck.delta_wno
wno = opacity_ck.wno
nwno = len(wno) 
bb , y2 , tp = set_bb(wno,delta_wno,nwno,ntmps,dt,tmin,tmax)
FOPI = np.zeros(nwno) + 1.0

print(ng,nt,nlevel)

flux_net_v_layer_full, flux_net_v_full, flux_plus_v_full, flux_minus_v_full , flux_net_ir_layer_full, flux_net_ir_full, flux_plus_ir_full, flux_minus_ir_full = climate(pressure, temp, dwni, bb , y2, tp, tmin, tmax, DTAU, TAU, W0, 
            COSB,ftau_cld, ftau_ray,GCOS2, DTAU_OG, TAU_OG, W0_OG, COSB_OG, W0_no_raman , surf_reflect, 
            ubar0,ubar1,cos_theta, FOPI, single_phase,multi_phase,frac_a,frac_b,frac_c,constant_back,constant_forward, tridiagonal , 
            wno,nwno,ng,nt, nlevel, ngauss, gauss_wts,True, True)



1 1 91


In [16]:
path = "../../research/EGP_paper/array/"


np.save(path+"BD_700_316_v_layer",flux_net_v_layer_full)
np.save(path+"BD_700_316_v_level",flux_net_v_full)
np.save(path+"BD_700_316_ir_layer",flux_net_ir_layer_full)
np.save(path+"BD_700_316_ir_level",flux_net_ir_full)





In [ ]:
tidal = -0.56687e-4*(Teff**4)
diff = np.abs((flux_net_ir_layer_full+tidal)/tidal)
plt.semilogy(diff)